In [ ]:
Célula de código <MTcGzA4oY8oz>
# %% [code]
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Parâmetros
start_time = datetime(2025, 11, 15, 15, 0, 0)  # 3h antes do evento
end_time = datetime(2025, 11, 16, 5, 0, 0)    # 3h depois do evento
interval_seconds = 10
camera_ids = [f'Camera_{i}' for i in range(1, 6)]  # 5 câmeras

# Criar timestamps em intervalos de 10 segundos
timestamps = pd.date_range(start=start_time, end=end_time, freq=f'{interval_seconds}s')

# Gerar id_frame
total_records = len(timestamps) * len(camera_ids)
id_frames = range(1, total_records + 1)

# DataFrame base expandindo para todas as câmeras
df = pd.DataFrame({
    'timestamp': np.repeat(timestamps, len(camera_ids)),
    'localizacao_camera': camera_ids * len(timestamps),
    'id_frame': id_frames
})

# Função para gerar eventos contínuos (True sequencial) aleatórios para cada tipo
def generate_events(series_length, max_event_duration, prob_event_start):
    # Inicialmente, tudo False
    events = np.zeros(series_length, dtype=bool)
    i = 0
    while i < series_length:
        if np.random.rand() < prob_event_start:
            duration = np.random.randint(6, max_event_duration)  # duração em número de frames (ex: 6*10s = 1min)
            end = min(i + duration, series_length)
            events[i:end] = True
            i = end
        else:
            i += 1
    return events

# Aplicar por câmera
def apply_events(df, max_event_duration, prob_event_start):
    event_column = []
    for cam in camera_ids:
        mask = df['localizacao_camera'] == cam
        length = mask.sum()
        event_col = generate_events(length, max_event_duration, prob_event_start)
        event_column.extend(event_col)
    return event_column

# Gerar eventos
np.random.seed(42)  # para reprodução

# Aglomeração: eventos até 30 min (180 frames), com chance moderada de começar
df['aglomeração'] = apply_events(df, max_event_duration=180, prob_event_start=0.02)

# Veículo parado: eventos até 10 min (60 frames), chance menor
df['veiculo_parado'] = apply_events(df, max_event_duration=60, prob_event_start=0.01)

# Pessoa caída: eventos até 5 min (30 frames), muito raros
df['pessoa_caida'] = apply_events(df, max_event_duration=30, prob_event_start=0.005)

# Mostrar as primeiras linhas para conferir
print(df.head())

# Salvar em CSV
df.to_csv('dados_sensor_urbano.csv', index=False)
print("Arquivo 'dados_sensor_urbano.csv' criado com sucesso!")

Saída da execução
1KB
	Stream
		timestamp localizacao_camera  id_frame  aglomeração  \
		0 2025-11-15 15:00:00           Camera_1         1        False
		1 2025-11-15 15:00:00           Camera_2         2        False
		2 2025-11-15 15:00:00           Camera_3         3        False
		3 2025-11-15 15:00:00           Camera_4         4        False
		4 2025-11-15 15:00:00           Camera_5         5        False

		   veiculo_parado  pessoa_caida
		0           False         False
		1           False         False
		2           False         False
		3           False         False
		4           False         False
		Arquivo 'dados_sensor_urbano.csv' criado com sucesso!

In [3]:
import pandas as pd

df = pd.read_csv('cameras_publicas.csv')

In [4]:
print(df.columns)


Index(['timestamp', 'localizacao_camera', 'id_frame', 'aglomeração',
       'veiculo_parado', 'pessoa_caida'],
      dtype='object')


In [9]:
import pandas as pd

# Carregar o DataFrame
cameras_publicas = pd.read_csv('cameras_publicas.csv')

# Derreter as colunas de eventos para criar coluna 'tipo_ocorrencia'
cameras_melted = cameras_publicas.melt(
    id_vars=['timestamp', 'localizacao_camera', 'id_frame'],
    value_vars=['aglomeração', 'veiculo_parado', 'pessoa_caida'],
    var_name='tipo_ocorrencia',
    value_name='ocorreu'
)

# Filtrar apenas ocorrências
ocorrencias = cameras_melted[cameras_melted['ocorreu'] == True].copy()

# Converter timestamp
ocorrencias['timestamp'] = pd.to_datetime(ocorrencias['timestamp'])

# Ordenar
ocorrencias = ocorrencias.sort_values(['localizacao_camera', 'tipo_ocorrencia', 'timestamp'])

# Definir limite de tempo para continuidade de evento
limite_tempo = pd.Timedelta(seconds=60)

# Função para identificar sequências
def identificar_sequencias(grupo):
    grupo = grupo.copy()
    grupo['delta'] = grupo['timestamp'].diff().fillna(pd.Timedelta(seconds=0))
    grupo['nova_seq'] = (grupo['delta'] > limite_tempo).cumsum()
    return grupo

# Aplicar
ocorrencias = ocorrencias.groupby(['localizacao_camera', 'tipo_ocorrencia']).apply(
    identificar_sequencias, include_groups=False).reset_index()


# Agrupar por sequências
duracoes = ocorrencias.groupby(['localizacao_camera', 'tipo_ocorrencia', 'nova_seq']).agg(
    inicio=('timestamp', 'min'),
    fim=('timestamp', 'max'),
    frames=('id_frame', 'count')
).reset_index()

# Calcular duração
duracoes['duracao_segundos'] = (duracoes['fim'] - duracoes['inicio']).dt.total_seconds()

# --- Consolidar todas as métricas em uma tabela só ---

tabela_resumo = duracoes.groupby(['localizacao_camera', 'tipo_ocorrencia']).agg(
    quantidade_eventos=('duracao_segundos', 'count'),
    duracao_media_segundos=('duracao_segundos', 'mean'),
    duracao_mediana_segundos=('duracao_segundos', 'median'),
    duracao_maxima_segundos=('duracao_segundos', 'max'),
    duracao_minima_segundos=('duracao_segundos', 'min'),
    total_duracao_segundos=('duracao_segundos', 'sum')
).reset_index()

# Exportar
tabela_resumo.to_csv('resumo_eventos.csv', index=False)

print('✅ Tabela consolidada criada e exportada como "resumo_eventos.csv"')


✅ Tabela consolidada criada e exportada como "resumo_eventos.csv"
